<a href="https://colab.research.google.com/github/Pavel184/PyTorch_basic_course/blob/Homework_lesson_6/Homework_lesson_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# попробуем запрограммировать простую рекурентную сеть. Возьмем датасет с прошлого занятия

import pandas as pd
import numpy as np
from datetime import datetime

import torch
from torchvision import transforms, datasets
import torch.nn as nn

from tqdm import tqdm_notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
a = torch.arange(4.)
a

tensor([0., 1., 2., 3.])

In [ ]:
torch.reshape(a, (2, 2))

tensor([[0., 1.],
        [2., 3.]])

In [ ]:
# Так как датасет очень большой, будем использовать данные за 2 последних года(2015-08-16 по 2017-08-15)
# Так же это имеет смысл, тк поведение покупателей могло изменяться год от года и слишком старые данные будут только мешать
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PyTorch/Lesson_6/data/kaggle/train.csv", 
                       parse_dates=['date'], skiprows= range(1,54043765)) 
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PyTorch/Lesson_6/data/kaggle/test.csv", parse_dates=['date'])
df_holidays_events = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PyTorch/Lesson_6/data/kaggle/holidays_events.csv")
df_items = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PyTorch/Lesson_6/data/kaggle/items.csv")
df_oil = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PyTorch/Lesson_6/data/kaggle/oil.csv", parse_dates=['date'])
df_stores = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PyTorch/Lesson_6/data/kaggle/stores.csv")
df_transactions = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PyTorch/Lesson_6/data/kaggle/transactions.csv")

In [ ]:
#df_oil = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PyTorch/Lesson_6/data/kaggle/oil.csv", parse_dates=['date'])
#df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PyTorch/Lesson_6/data/kaggle/test.csv", parse_dates=['date'])

In [ ]:
df_train.head(3)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,54043764,2015-08-16,1,103665,1.0,False
1,54043765,2015-08-16,1,105574,3.0,False
2,54043766,2015-08-16,1,105575,7.0,False


In [ ]:
df_train.tail(3)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
71453273,125497037,2017-08-15,54,2110456,192.0,False
71453274,125497038,2017-08-15,54,2113914,198.0,True
71453275,125497039,2017-08-15,54,2116416,2.0,False


In [ ]:
#df_train['date'] = pd.to_datetime(df_train['date'])

In [ ]:
#df_train.loc[df_train['date']==pd.datetime(2015,8,16)]

In [ ]:

#df_train = df_train.loc[(df_train['date']>pd.datetime(2015,8,16)) & (df_train['date']<=pd.datetime(2017,8,16))]

In [ ]:
df_test.head(3)

,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,False
1,125497041,2017-08-16,1,99197,False
2,125497042,2017-08-16,1,103501,False


In [ ]:
df_holidays_events.head(3)

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False


In [ ]:
df_items.head(3)

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0


In [ ]:
df_oil.head(30) # есть пропущенные значения, но судя по датасету они должны равняться ~ значениям вокруг, используем метод .bfill()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21
6,2013-01-09,93.08
7,2013-01-10,93.81
8,2013-01-11,93.60
9,2013-01-14,94.27


In [ ]:
df_oil = df_oil.fillna(method='bfill')

In [ ]:
df_stores.head(3)

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8


In [ ]:
df_transactions.head(3)

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358


In [ ]:
df_train = pd.merge(df_train,df_items,on='item_nbr')
df_test = pd.merge(df_test,df_items, on='item_nbr')

In [ ]:
df_train = pd.merge(df_train, df_stores, on='store_nbr')
df_test = pd.merge(df_test,df_stores, on='store_nbr')

In [ ]:
df_train = pd.merge(df_train, df_oil, on='date')
df_test = pd.merge(df_test,df_oil, on='date')

In [ ]:
df_train.head(3)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster,dcoilwtico
0,54129625,2015-08-17,1,103665,1.0,False,BREAD/BAKERY,2712,1,Quito,Pichincha,D,13,41.93
1,54129626,2015-08-17,1,105574,2.0,False,GROCERY I,1045,0,Quito,Pichincha,D,13,41.93
2,54129627,2015-08-17,1,105575,13.0,False,GROCERY I,1045,0,Quito,Pichincha,D,13,41.93


In [ ]:
def preproc_test_train(df):
  df['date'] = pd.to_datetime(df['date'])
  df['dayofweek'] = [d.dayofweek for d in df["date"]] 
  df['dayofyear'] = [d.dayofyear for d in df["date"]]
  df['year'] = [d.year for d in df["date"]] 
  df['month'] = [d.month for d in df["date"]]
  df['time_delta'] = (datetime.strptime('2000/12/31', "%Y/%m/%d") - df['date']).dt.days.abs()
  return df

df_train = preproc_test_train(df_train)

In [ ]:
column_list = ['family', 'city', 'state', 'type', 'item_nbr', 'class', 'store_nbr', 'cluster', 'onpromotion']
def cat2num(df, column_list):
  for name in column_list:
    df[name] = df[name].astype('category').cat.codes
  return df

df_train = cat2num(df_train, column_list)

In [ ]:
df_train.head(3)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster,dcoilwtico
0,54129625,2015-08-17,0,4,1.0,0,5,184,1,18,12,3,12,41.93
1,54129626,2015-08-17,0,5,2.0,0,12,30,0,18,12,3,12,41.93
2,54129627,2015-08-17,0,6,13.0,0,12,30,0,18,12,3,12,41.93


In [ ]:
#df_train.to_csv("/content/drive/MyDrive/Colab Notebooks/PyTorch/Lesson_6/data/kaggle/df_train.csv", index=True)

In [ ]:
#df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PyTorch/Lesson_6/data/kaggle/df_train.csv')

In [ ]:
validation = df_train.loc[df_train['date']>pd.datetime(2017,2,15)]

train = df_train.loc[df_train['date']<=pd.datetime(2017,2,15)] 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
target = ['unit_sales']

In [ ]:
train.drop(['date'], axis=1, inplace=True) 
validation.drop(['date'], axis=1, inplace=True) 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
train.drop(['id'], axis=1, inplace=True) 
validation.drop(['id'], axis=1, inplace=True) 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
train.head(3)

,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,type,cluster,dcoilwtico
0,0,4,1.0,0,5,184,1,18,12,3,12,41.93
1,0,5,2.0,0,12,30,0,18,12,3,12,41.93
2,0,6,13.0,0,12,30,0,18,12,3,12,41.93


In [ ]:
from functools import lru_cache

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, used_length):
      self._X = dataset.drop(target, axis = 1).values.astype(np.float32)
      self._y = dataset[target].values.astype(np.float32)
      self._length = used_length
    
    def __len__(self):
        return len(self._X) 

    def __getitem__(self,idx):
        length = self._length
        X = self._X[idx:(idx+length), : ]
        X = torch.tensor(X)
        #self._X.loc[i:i+n; : ]
        #X = torch.reshape(X, (self._length, X.shape[0]))
        y = self._y[idx:(idx+length), : ]
        y = torch.tensor(y)
        return X, y#, length 

In [ ]:
train_dataset = MyDataset(train, 256)
valid_dataset = MyDataset(validation, 256)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=256,
                          shuffle=False,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=256,
                          shuffle=False,
                          num_workers=1)

In [ ]:
for i in train_loader:
  print(i[0].shape)
  print(i[1].shape)
  #print(i[2])
  print(i[0])
  print(i[1])

  break

torch.Size([256, 256, 11])
torch.Size([256, 256, 1])
tensor([[[   0.0000,    4.0000,    0.0000,  ...,    3.0000,   12.0000,
            41.9300],
         [   0.0000,    5.0000,    0.0000,  ...,    3.0000,   12.0000,
            41.9300],
         [   0.0000,    6.0000,    0.0000,  ...,    3.0000,   12.0000,
            41.9300],
         ...,
         [   0.0000,  649.0000,    0.0000,  ...,    3.0000,   12.0000,
            41.9300],
         [   0.0000,  650.0000,    0.0000,  ...,    3.0000,   12.0000,
            41.9300],
         [   0.0000,  652.0000,    0.0000,  ...,    3.0000,   12.0000,
            41.9300]],

        [[   0.0000,    5.0000,    0.0000,  ...,    3.0000,   12.0000,
            41.9300],
         [   0.0000,    6.0000,    0.0000,  ...,    3.0000,   12.0000,
            41.9300],
         [   0.0000,   12.0000,    0.0000,  ...,    3.0000,   12.0000,
            41.9300],
         ...,
         [   0.0000,  650.0000,    0.0000,  ...,    3.0000,   12.0000,
         

In [ ]:
class LSTMFixedLen(nn.Module) :
    def __init__(self, input_size, hidden_dim) :
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(lstm_out)
    
lstm_init = LSTMFixedLen(11, 32)
optimizer = torch.optim.Adam(lstm_init.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [ ]:
for epoch in tqdm_notebook(range(10)):  
    lstm_init.train()
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]
        optimizer.zero_grad()
        outputs = lstm_init(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    lstm_init.eval()
    loss_accumed = 0
    for X, y, _ in valid_loader:
        output = lstm_init(X)
        loss = criterion(output, y)
        loss_accumed += loss
    print("Epoch {} valid_loss {}".format(epoch, loss_accumed))

print('Training is finished!')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/10 [00:00<?, ?it/s]